In [10]:
from tesseraclassifier import habitat_classifier
from tesseraclassifier import classifier_utils, viz_utils
import geopandas as gpd
import tempfile
from IPython.display import display

import importlib
importlib.reload(classifier_utils)
importlib.reload(habitat_classifier)


<module 'tesseraclassifier.habitat_classifier' from '/home/jk871/tessera-classification/src/tesseraclassifier/habitat_classifier.py'>

### Specify where the input labels are, and the output for the classification

In [2]:
roi_name = "Cambridge"

labels_json_file = f'/home/jk871/tessera-interactive-map/cambridge_labels.json'
output_dir = f"/home/jk871/cci_workshop/classification_outputs/{roi_name}"

In [3]:
labels = classifier_utils.load_training_labels_from_json(labels_json_file)
hc = habitat_classifier.HabitatClassifier(labels, 2024)
hc.train()


--- Step 1: Preparing Training Data ---
Fetching list of all available tiles...
Found 146452 total available tiles             for 2024.

Processing point 1/36 at                     (52.1439, 0.0907)...

Processing point 2/36 at                     (52.1443, 0.1440)...

Processing point 3/36 at                     (52.1398, 0.1229)...

Processing point 4/36 at                     (52.1409, 0.1313)...

Processing point 5/36 at                     (52.1411, 0.1337)...

Processing point 6/36 at                     (52.1370, 0.1060)...

Processing point 7/36 at                     (52.1343, 0.1039)...

Processing point 8/36 at                     (52.1294, 0.1093)...

Processing point 9/36 at                     (52.1316, 0.1089)...

Processing point 10/36 at                     (52.1308, 0.1269)...

Processing point 11/36 at                     (52.1292, 0.1308)...

Processing point 12/36 at                     (52.1253, 0.1283)...

Processing point 13/36 at                     (52.1241

In [5]:
# Classification dry run - just to estimate the size of each ROI
hc.classify_tiles(None, hc.training_tiles, dry_run=True)
hc.classify_region(None, classifier_utils.roi_from_points(labels, 30), dry_run=True)

Found 2 tiles to process.
Projecting points to a suitable local CRS: EPSG:32631
Found 78 tiles to process.


In [7]:
def run_classification(buffer_in_km):
    key = "bounding_box"
    temp_dir = tempfile.mkdtemp(prefix=f"tessera_classify_{roi_name}_{key}_")
    hc.classify_region(temp_dir, classifier_utils.roi_from_points(labels, buffer_in_km), max_workers=50)
    hc.mosaic_classification(temp_dir, f"{output_dir}/{key}.tif", cleanup=False)

In [8]:
run_classification(30)

Projecting points to a suitable local CRS: EPSG:32631
Found 78 tiles to process.

Starting parallel classification using up to 50 CPU cores...



Parallel processing complete. 78 of 78 tiles were successfully classified.
Intermediate classified tiles saved in: /tmp/tessera_classify_Cambridge_bounding_box_oducg13g
LALAL
False
Using EPSG:32630 as the common projection for stitching.
Starting parallel reprojection of 78 tiles...

Stitching 78 aligned tiles into final map...
✅ Successfully created final classification map: /home/jk871/cci_workshop/classification_outputs/Cambridge/bounding_box.tif


In [13]:
# Define paths to your data
key = "bounding_box"
CLASSIFIED_TILE = f"{output_dir}/{key}.tif"
label_to_id, id_to_label, class_colors = classifier_utils.load_visualization_mappings(labels_json_file)

interactive_map = viz_utils.create_interactive_map(
    classified_raster_path=CLASSIFIED_TILE,
    training_points=labels,
    id_to_label=id_to_label,
    class_colors_hex=class_colors
)

display(interactive_map)

--- Loading Mappings for Visualization ---
Generated class mappings from all labels in the file.
Loaded 4 color definitions.
✅ Mappings ready for visualization.
Reprojecting classified raster to Web Mercator...
Converting warped raster to displayable image...
✅ Interactive map with legend created.
